<a href="https://colab.research.google.com/github/DSookarah/CE888_Assignment/blob/main/CE888_Preprocessing.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [4]:
# Installing and importing all necessary librairies 

!pip install tweet-preprocessor 2>/dev/null 1>/dev/null

!pip install transformers
import preprocessor as p
import numpy as np 
import pandas as pd 

In [5]:
# Importing the datasets (e.g. emotion) and then assigning headers to the columns

train_data=pd.read_csv('https://raw.githubusercontent.com/cardiffnlp/tweeteval/main/datasets/emotion/train_text.txt', delimiter='\t') 
test_data=pd.read_csv('https://raw.githubusercontent.com/cardiffnlp/tweeteval/main/datasets/emotion/test_text.txt', delimiter='\t') 
train_labels= pd.read_csv('https://raw.githubusercontent.com/cardiffnlp/tweeteval/main/datasets/emotion/train_labels.txt', delimiter='\t') 
test_labels= pd.read_csv('https://raw.githubusercontent.com/cardiffnlp/tweeteval/main/datasets/emotion/test_labels.txt', delimiter='\t') 
train_data.columns=['Tweet']
test_data.columns=['Tweet']
train_labels.columns=['Emotion']
test_labels.columns=['Emotion']
#test_labels.head(10)

In [6]:
# Using the aspell txt file for correcting the misspelling of words in the tweets
misspell_data = pd.read_csv("/aspell.txt",sep=":",names=["correction","misspell"])
misspell_data.misspell = misspell_data.misspell.str.strip()
misspell_data.misspell = misspell_data.misspell.str.split(" ")
misspell_data = misspell_data.explode("misspell").reset_index(drop=True)
misspell_data.drop_duplicates("misspell",inplace=True)
miss_corr = dict(zip(misspell_data.misspell, misspell_data.correction))

# The misspelled correction function to correct the words

def misspelled_correction(val):
    for x in val.split(): 
        if x in miss_corr.keys(): 
            val = val.replace(miss_corr[x],x) 
    return val

train_data["cleaned"] = train_data.Tweet.apply(lambda x : misspelled_correction(x))
test_data["cleaned"] = test_data.Tweet.apply(lambda x : misspelled_correction(x))
#test_data.head(10)

In [7]:
# Getting rid of contractions

contractions = pd.read_csv("/contractions.csv")
cont_dic = dict(zip(contractions.Contraction, contractions.Meaning))

# Defining function to change contractions

def cont_to_meaning(val): 
  
    for x in val.split(): 
        if x in cont_dic.keys(): 
            val = val.replace(x, cont_dic[x]) 
    return val

# Expanding uppercase and lowercase contractions

train_data.cleaned = train_data.cleaned.apply(lambda x : cont_to_meaning(x))
test_data.cleaned = test_data.cleaned.apply(lambda x : cont_to_meaning(x))
#test_data.head(10)
train_data['cleaned']=train_data['cleaned'].apply(lambda x: x.lower())
test_data['cleaned']=test_data['cleaned'].apply(lambda x: x.lower())
train_data.cleaned = train_data.cleaned.apply(lambda x : cont_to_meaning(x))
test_data.cleaned = test_data.cleaned.apply(lambda x : cont_to_meaning(x))
#train_data.head(10)

In [8]:
# Getting rid of user mentions and URLs

p.set_options(p.OPT.MENTION, p.OPT.URL)
train_data["cleaned"]=train_data.cleaned.apply(lambda x : p.clean(x))
test_data["cleaned"]=test_data.cleaned.apply(lambda x : p.clean(x))
#train_data.head(10)


# Defining the function that will be used to remove punctuation

def punctuation(val): 
  
    punctuations = '''?!()-[]{};:'"\,<>./@#$%^&_~'''
  
    for x in val.lower(): 
        if x in punctuations: 
            val = val.replace(x, " ") 
    return val
train_data["cleaned"]=train_data.cleaned.apply(lambda x : punctuation(x))
test_data["cleaned"]=test_data.cleaned.apply(lambda x : punctuation(x))


In [9]:
# Checking the final output of the 'cleaned' data
test_data.head(10)

,Tweet,cleaned
0,@user Interesting choice of words... Are you c...,interesting choice of words are you confirm...
1,My visit to hospital for care triggered #traum...,my visit to hospital for care triggered traum...
2,@user Welcome to #MPSVT! We are delighted to h...,welcome to mpsvt we are delighted to have yo...
3,What makes you feel #joyful?,what makes you feel joyful
4,i am revolting.,i am revolting
5,Rin might ever appeared gloomy but to be a mel...,rin might ever appeared gloomy but to be a mel...
6,In need of a change! #restless,in need of a change restless
7,@user @user #cmbyn does screen August 4 &amp;...,cmbyn does screen august 4 amp 6 at miff
8,@user Get Donovan out of your soccer booth. He...,get donovan out of your soccer booth he is aw...
9,@user how can u have sold so many copies but u...,how can u have sold so many copies but ur game...
